<div id="top">包括以下操作：</div>
<li>运行文件</li>
<li><a href="#1">inference.py 代码解析</a></li>
<li><a href="#2">获取各类别预测准确率</a></li>
<li><a href="#3">学习率</a></li>
<li><a href="#4">断点续训</a></li>

代码里 如：args.include_index, 一般在 parser.add_argument 处就是 --include-index，一个是下划线，一个是小短线。

### 运行文件

文件运行：https://huggingface.co/docs/timm/training_script

#### 运行 train.py 文件

In [ ]:
    # CUDA_VISIBLE_DEVICES='0,1,2,3' python -m torch.distributed.launch --master_port=29508 --nproc_per_node=4 train.py \
    # --data=/App/chenbowen/dataset/eggDataset/dataset \
    # --initial-checkpoint='' \
    # --num-classes=10 \
    # --img-size=552 \
    # --lr=0.05 \
    # --output=chicken_eggs_version_6 \
    # --batch-size=8 \
    # --epochs=100 \
    # --model=efficientnet_b3

#### 运行 inference.py 文件

In [ ]:
    # python validate.py \
    # --dataset=/App/chenbowen/dataset/newEggDataset/test \
    # --num-classes=10 \
    # --img-size=552 \
    # --batch-size=5 \
    # --model=efficientnet_b3 \
    # --pretrained=/App/chenbowen/pytorch-image-models-main/chicken_eggs_version_5/20240823-004244-efficientnet_b3-552/model_best.pth.tar

### <a id="1">inference.py 代码解析</a>

<a href='#top'>▲ Top</a>

for batch_idx, (input, _) in enumerate(loader): ， 打印 _ , 得到的就是真实标签。以 tensor 的形式展现。

![image.png](./inference/1-1.png)

取 target.tolist() 就得到下面的：

![image.png](./inference/1-2.png)

`.softmax(-1)` 是一个用于计算张量最后一个维度上的 softmax 函数的操作。softmax 函数用于将向量的每个元素转换为介于 0 和 1 之间的概率值，并使这些概率值的总和等于 1。

![image.png](./inference/2.png)

![image.png](./inference/3.png)

In [ ]:
    all_indices = np.concatenate(all_indices, axis=0) if all_indices else None # 记录所有的标签

    all_outputs = np.concatenate(all_outputs, axis=0).astype(np.float32) # 记录所有最大概率是多少

PyTorch 获取 Tensor 值的三种方法：item()、tolist() 与 .numpy(): https://cloud.baidu.com/article/2677107

### <a id="2">获取各类别预测准确率</a>

<a href='#top'>▲ Top</a>

In [ ]:
    for i in range(10):  # 种类的数量
        denominator = list(true_labels).count(i)     # 从list里获取i的数量，得到各类的样本数量
        numerator = np.equal(true_labels, i) & np.equal(all_indices, i)  # 分别从真值、预测值中筛选目标值,并与计算 得出正确的个体
        numerator = np.sum(numerator==1)        # 统计所有正确的个体
 
        print('Accuracy of {:2d}: {:.5f}'.format(i,numerator/denominator))

### <a id="3">学习率</a>

<a href='#top'>▲ Top</a>

1.771e-02 是科学计数法表示的数字，它可以转换为常规的数字形式。在这种表示法中，`e-02` 表示将小数点向左移动两位，即乘以 10 的负二次方。因此，1.771e-02 实际上等于 0.01771。

所以，1.771e-02 表示的数字是 0.01771。这种表示法通常用于表示非常小的数值或在科学计算中处理较小的数值。

### <a id="4">断点续训</a>

<a href='#top'>▲ Top</a>

In [ ]:
    # 使用 resume 进行断点续训 

    CUDA_VISIBLE_DEVICES='0,1,2,3' python -m torch.distributed.launch --master_port=29508 --nproc_per_node=4 train.py \
    --data=/App/chenbowen/dataset/colorClassifyData/dataset \
    --initial-checkpoint='' \
    --num-classes=3 \
    --img-size=552 \
    --lr=0.05 \
    --output=chicken_eggs_version_color \
    --batch-size=16 \
    --resume=/App/chenbowen/pytorch-image-models-main/chicken_eggs_version_color/20240920-021305-efficientnet_b3-552/last.pth.tar \
    --epochs=100 \
    --model=efficientnet_b3